Version 12
Dated 15 Nov.

In this version we have changed the code to look for the most frequent words in the first fifty results of patents from a list of 132 results. The results are directly from the USTrademark office website. We will be moving into a more elaborate use of Natural Language Processing Tool Kit in python to better assess the frequency of words used in the first fifty results we are scraping from the UStrademark Office's website. This code will be modified further.


In [1]:
# import libraries
#import urllib2
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
from random import randint

In [2]:
def getsoupfromlink(quote_page):
    # specify the url
    quote_page = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=2&f=S&l=50&Query=ACLM%2Fpython+&d=PTXT'
    # query the website and return the html to the variable ‘page’
    page = urlopen(quote_page)
    # parse the html using beautiful soap and store in variable `soup`
    soup = BeautifulSoup(page, 'html.parser')
    return soup

#soup = getsoupfromlink(quote_page)

soup

In [3]:
#s = soup.text

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def find_between_r( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""


#print (find_between( s, "What is claimed is:", "* * * * *" ))
#print (find_between_r (s, '123', 'abc' ))

In [4]:
def extractlink (soup):
    alllinks = []
    count = 0 
    for link in soup.find_all('a', href=True):
        x =  link['href']
        if x[:19]=='/netacgi/nph-Parser' and x.find('Next') == -1:

            count = count+1
            alllinks.append ('http://patft.uspto.gov'+x)
    return alllinks

#alllinks = extractlink(soup)


In [5]:
def remove_duplicates(l):
    return list(set(l))

#alllinks = remove_duplicates(alllinks)

#print (len(alllinks))

In [10]:
def getdatafromlink(link, lstofp):
    from urllib.request import urlopen
    from bs4 import BeautifulSoup
    #quote_page2 = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=1&p=1&f=G&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&OS=ACLM/python&RS=ACLM/python/'
    quote_page2 = link
    
    page2 = urlopen(quote_page2)

    sleep(randint(8,15)/30)
    soupinfun = BeautifulSoup(page2, 'html.parser')
    
    try:
        heading= (soupinfun.find('font', {'size':'+1'}).text).replace("\n", "")
        heading = ' '.join(heading.split())
        abstract= (soupinfun.p.text).replace("\n", "")
        abstract = ' '.join(abstract.split())
        claims = (find_between (soupinfun.text, 'What is claimed is:', '* * * * *' )).replace("\n", "")
        claims = ' '.join(claims.split())
        #print soup
        lstofp.append([link,heading, abstract, claims])
        page2.close()
        #time.sleep(.1)

    except Exception:
        pass
    return lstofp;



In [ ]:
def createdf(alllinks):
    listofp =[]
    for item in alllinks:
        listofp = getdatafromlink(item,listofp)

    cols = ['Link', 'Heading', 'Abstract', 'Claims']

    df1 = pd.DataFrame(listofp, columns=cols)

    df1.to_csv('output.txt', sep='|')
    return df1

#df1 = createdf(alllinks)

In [ ]:
quote_page = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=2&f=S&l=50&Query=ACLM%2Fpython+&d=PTXT'

listofresultpages=[]

for num in range(1,9):
    leftlink= 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p='
    midnumber = num
    rightlink = '&f=S&l=50&Query=ACLM%2Fpython+&d=PTXT'
    quotepagejoined = leftlink+str(midnumber)+rightlink
    listofresultpages.append([quotepagejoined])

#print (quotepagejoined == quote_page)

#listofresultpages

In [11]:
quote_page = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=2&f=S&l=50&Query=ACLM%2Fpython+&d=PTXT'

for page in listofresultpages:
    soup = getsoupfromlink(page)
    alllinks = extractlink(soup)
    alllinks = remove_duplicates(alllinks)
    #print (alllinks)
    df1 = createdf(alllinks)
    print (df1)



                                                 Link  \
0   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
1   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
2   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
3   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
4   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
5   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
6   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
7   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
8   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
9   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
10  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
11  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
12  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
13  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
14  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
15  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
16  http://patft.uspto.gov/neta

KeyboardInterrupt: 

In [ ]:
randint(8,15)/100

In [26]:
quote_page =['http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=1&f=S&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&Page=Prev&OS=ACLM/python&RS=ACLM/python', 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=2&f=S&l=50&Query=ACLM%2Fpython+&d=PTXT'
,'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=3&f=S&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&Page=Next&OS=ACLM/python&RS=ACLM/python']

dfmerged = pd.DataFrame()


for quote in quote_page:
    soup = getsoupfromlink(quote)
    alllinks = extractlink(soup)
    alllinks = remove_duplicates(alllinks)
    #print (alllinks)
    x = createdf(alllinks)
    #print (df1)
    x = x.reset_index(drop=True)
    #dfmerged = dfmerged.add(x)
    dfmerged = pd.concat([dfmerged, x], ignore_index = True)

print (dfmerged)

                                                  Link  \
0    http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
1    http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
2    http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
3    http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
4    http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
5    http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
6    http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
7    http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
8    http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
9    http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
10   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
11   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
12   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
13   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
14   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
15   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
16   http://pa

In [25]:
for quote in quote_page:
    print(quote)

http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=1&f=S&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&Page=Prev&OS=ACLM/python&RS=ACLM/python
http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=2&f=S&l=50&Query=ACLM%2Fpython+&d=PTXT
http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=3&f=S&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&Page=Next&OS=ACLM/python&RS=ACLM/python


In [ ]:
quote_page[1]

In [ ]:

soup = getsoupfromlink(quote_page[1])
alllinks = extractlink(soup)
alllinks = remove_duplicates(alllinks)
#print (alllinks)
df1 = createdf(alllinks)
print (df1)


In [21]:
dfmerged.to_csv("test.txt", sep = "|", encoding='utf-8')


In [ ]:
from collections import Counter

def splitplay (test):
    x = Counter(test.split()).most_common(1)
    return x[0][0]

dfmerged['freqofwords'] = dfmerged['Abstract'].apply(splitplay)

dfmerged['freqofwords'] 


In [28]:
dfmerged #= dfmerged.add(x)


,Link,Heading,Abstract,Claims
0,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,Replacement of build to order parts with post ...,An embedded imaging system for addressing burn...,1. A method for installing a software onto an ...
1,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,Pattern modeling methods and systems,"In a computer-implemented method, models are d...",1. A computer-implemented method comprising: d...
2,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,System for translating diverse programming lan...,A computer language translator that translates...,1. A computer language translator for translat...
3,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,Client program executable on multiple heteroge...,A device receives a program that includes one ...,1. A computing device-implemented method compr...
4,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,"Systems, methods and apparatus for modeling, s...","Systems, methods and apparatus are provided th...",
5,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,System and a method for web-based editing of d...,A method and a system for editing documents on...,1. A system for editing both a webpage documen...
6,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,Garbage collection implemented in hardware,A computing device is provided and includes a ...,"1. A computing device, comprising: a memory mo..."
7,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,Method and system for data centric heap profiling,A method and system for data centric heap prof...,"1. A method, implemented in a computing device..."
8,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,Optimizing program by reusing execution result...,A technique for optimizing a program by reusin...,"1. A method for optimizing a program, comprisi..."
9,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,Dynamic centralized unit determination in a cr...,"A charging system includes a rating engine, a ...","1. A credit control server, comprising: a rati..."


In [29]:
dfmerged.Link.value_counts()


http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=58&p=2&f=G&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&OS=ACLM/python&RS=ACLM/python     3
http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=71&p=2&f=G&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&OS=ACLM/python&RS=ACLM/python     3
http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=72&p=2&f=G&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&OS=ACLM/python&RS=ACLM/python     3
http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=73&p=2&f=G&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&OS=ACLM/python&RS=ACLM/python     3
http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=70&p=2&f=G&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&OS=ACLM/python&RS=ACLM/python     3
http://pat